# Load data

In [8]:
from lxml import etree
import os

# Correct file path – make sure this file really lives in your notebook's CWD
file_path = "data/Faust._Der_Tragoedie_zweiter_Teil.11d12.0.xml"

# Parse XML
tree = etree.parse(file_path)
root = tree.getroot()

# Check what tags exist
tags = set(elem.tag for elem in root.iter())
for tag in tags:
    print(tag)

{http://www.tei-c.org/ns/1.0}title
{http://www.tei-c.org/ns/1.0}biblFull
{http://www.tei-c.org/ns/1.0}teiHeader
{http://www.tei-c.org/ns/1.0}div
{http://www.tei-c.org/ns/1.0}langUsage
{http://www.tei-c.org/ns/1.0}publicationStmt
{http://www.tei-c.org/ns/1.0}titleStmt
{http://www.tei-c.org/ns/1.0}body
{http://www.tei-c.org/ns/1.0}creation
{http://www.tei-c.org/ns/1.0}term
{http://www.tei-c.org/ns/1.0}language
{http://www.tei-c.org/ns/1.0}notesStmt
{http://www.tei-c.org/ns/1.0}TEI
{http://www.tei-c.org/ns/1.0}name
{http://www.tei-c.org/ns/1.0}pb
{http://www.tei-c.org/ns/1.0}resp
{http://www.tei-c.org/ns/1.0}profileDesc
{http://www.tei-c.org/ns/1.0}lg
{http://www.tei-c.org/ns/1.0}respStmt
{http://www.tei-c.org/ns/1.0}ref
{http://www.tei-c.org/ns/1.0}sourceDesc
{http://www.tei-c.org/ns/1.0}idno
{http://www.tei-c.org/ns/1.0}fileDesc
{http://www.tei-c.org/ns/1.0}desc
{http://www.tei-c.org/ns/1.0}hi
{http://www.tei-c.org/ns/1.0}keywords
{http://www.tei-c.org/ns/1.0}date
{http://www.tei-c.org/

In [10]:

# Register TEI namespace
ns = {'tei': "http://www.tei-c.org/ns/1.0"}# Check all div types
divs = tree.xpath("//tei:div", namespaces=ns)
types = set(div.get("type") for div in divs if div.get("type") is not None)

print("All div types found:", types)


All div types found: {'h4', 'act', 'text', 'front'}


# Separate data extraction


In [11]:
# build a list of only those <div>s you want to call “Scene”
scene_divs = [d for d in divs if d.find("tei:head", namespaces=ns) is not None]
scenes = []
for i, div in enumerate(scene_divs, start=1):
    head = div.find("tei:head", namespaces=ns)
    scenes.append(head)
for scene in scenes:
    print("Scene:", scene.text)


Scene: Der Tragödie zweiter Teil.
Scene: Anmutige Gegend.
Scene: Kaiserliche Pfalz. Saal des Thrones.
Scene: Weitläufiger Saal mit Nebengemächern
Scene: Lustgarten.
Scene: Finstere Galerie.
Scene: Hell erleuchtete Säle.
Scene: Rittersaal.
Scene: Hochgewölbtes enges gotisches Zimmer
Scene: Laboratorium
Scene: Pharsalische Felder.
Scene: Am oberen Peneios.
Scene: Am unteren Peneios.
Scene: Am obern Peneios.
Scene: Felsbuchten des ägäischen Meers.
Scene: Vor dem Palaste des Menelas zu Sparta.
Scene: Innerer Burghof
Scene: [Schattiger Hain].
Scene: Hochgebirg.
Scene: Auf dem Vorgebirg.
Scene: Des Gegenkaisers Zelt.
Scene: Offene Gegend.
Scene: Palast.
Scene: Tiefe Nacht.
Scene: Mitternacht.
Scene: Großer Vorhof des Palasts.
Scene: Grablegung.
Scene: Bergschluchten.


In [ ]:
speakers = tree.xpath("//tei:sp/tei:speaker", namespaces=ns)
print(f"Found {len(speakers)} speakers.")
for speaker in speakers:
    print("Speaker:", speaker.text)

Found 902 speakers.
Speaker: FAUST.
Speaker: GEIST.
Speaker: FAUST
Speaker: GEIST.
Speaker: FAUST.
Speaker: GEIST.
Speaker: FAUST.
Speaker: GEIST.
Speaker: FAUST.
Speaker: GEIST.
Speaker: FAUST
Speaker: WAGNER.
Speaker: FAUST.
Speaker: WAGNER.
Speaker: FAUST.
Speaker: WAGNER.
Speaker: FAUST.
Speaker: WAGNER.
Speaker: FAUST.
Speaker: WAGNER.
Speaker: FAUST.
Speaker: WAGNER.
Speaker: FAUST.
Speaker: WAGNER.
Speaker: FAUST
Speaker: CHOR DER ENGEL.
Speaker: FAUST.
Speaker: CHOR DER WEIBER.
Speaker: CHOR DER ENGEL.
Speaker: FAUST.
Speaker: CHOR DER JÜNGER.
Speaker: CHOR DER ENGEL.
Speaker: EINIGE HANDWERKSBURSCHEN.
Speaker: ANDRE.
Speaker: DIE ERSTEN.
Speaker: EIN HANDWERKSBURSCH.
Speaker: ZWEITER.
Speaker: DIE ZWEITEN.
Speaker: EIN DRITTER.
Speaker: VIERTER.
Speaker: FÜNFTER.
Speaker: DIENSTMÄDCHEN.
Speaker: ANDRE.
Speaker: ERSTE.
Speaker: ANDRE.
Speaker: SCHÜLER.
Speaker: BÜRGERMÄDCHEN.
Speaker: ZWEITER SCHÜLER
Speaker: ERSTER.
Speaker: BÜRGER.
Speaker: BETTLER
Speaker: ANDRER BÜRGER.
Spe

In [ ]:
lines = tree.xpath("//tei:l", namespaces=ns)
for line in lines[:10]:
    print("Line:", line.text)


Line: Habe nun, ach! Philosophie,
Line: Juristerei und Medizin,
Line: Und leider auch Theologie
Line: Durchaus studiert, mit heißem Bemühn.
Line: Da steh' ich nun, ich armer Tor,
Line: Und bin so klug als wie zuvor!
Line: Heiße Magister, heiße Doktor gar,
Line: Und ziehe schon an die zehen Jahr'
Line: Herauf, herab und quer und krumm
Line: Meine Schüler an der Nase herum –


In [ ]:
lines_text = [line.text for line in lines if line.text is not None]
print(f"Total lines with text: {len(lines_text)}")

Total lines with text: 4334


In [ ]:
import pandas as pd
records = []
for scene in scenes:
    scene_id = scene.get('xml:id')
    for sp in scene.xpath('.//sp'):
        speaker = sp.xpath('./speaker/text()')[0]
        for line in sp.xpath('./l'):
            text = line.text.strip()
            records.append({'scene': scene_id, 'speaker': speaker, 'line': text})
df = pd.DataFrame(records)

# Rhyme Scheme Detection

In [ ]:
lines_text = [line.text for line in lines]
# Remove None or empty lines
lines_text = [line for line in lines_text if line and line.strip()]

last_words = []
for line in lines_text:
    tokens = line.split()
    # look from the end for the first non-punctuation word
    while True:
    # pop tokens until a non-punctuation word is found
        tok = tokens.pop() 
        word = tok.strip('.,;:!?…–—-"\'()[]')
        if word:
            last_words.append(word)
            break


def get_rhyme_part(word):
    return word[-3:].lower()

rhyme_parts = [get_rhyme_part(word) for word in last_words]
print("Rhyme parts:", rhyme_parts)



Rhyme parts: ['hie', 'zin', 'gie', 'ühn', 'tor', 'vor', 'gar', 'ahr', 'umm', 'rum', 'nen', 'nen', 'fen', 'fen', 'fel', 'fel', 'sen', 'sen', 'ren', 'ren', 'eld', 'elt', 'ben', 'ben', 'und', 'und', 'eiß', 'eiß', 'elt', 'ält', 'men', 'men', 'ein', 'cht', 'cht', 'ier', 'mir', 'öhn', 'ehn', 'ben', 'ben', 'den', 'den', 'och', 'och', 'cht', 'cht', 'auf', 'ckt', 'auf', 'ckt', 'llt', 'pft', 'pft', 'elt', 'erz', 'mmt', 'erz', 'mmt', 'tur', 'ein', 'nur', 'ein', 'and', 'uch', 'and', 'nug', 'auf', 'auf', 'ist', 'ier', 'ärt', 'mir', 'ört', 'ick', 'nen', 'ück', 'nen', 'ieb', 'len', 'len', 'ieb', 'len', 'cht', 'gen', 'gen', 'cht', 'sen', 'tot', 'sen', 't!‹', 'ebt', 'ebt', 'gen', 'hen', 'gen', 'gen', 'gen', 'nur', 'tur', 'ens', 'ngt', 'ngt', 'ein', 'her', 'her', 'ein', 'gen', 'gen', 'gen', 'gen', 'mir', 'cht', 'det', 'len', 'eht', 'rab', 'an', 'ist', 'ich', 'ißt', 'len', 'len', 'ben', 'ben', 'mir', 'cht', 'gen', 'gen', 'nun', 'cht', 'uen', 'ehn', 'ehn', 'uen', 'ruf', 'huf', 'ben', 'ang', 'ang', 'ert', 

# Database preparation

In [ ]:
import spacy
# Load German language model
nlp = spacy.load("de_core_news_sm")
# Create a set of stopwords
stopwords = set(nlp.Defaults.stop_words)

In [ ]:
from collections import Counter

# collect speaker stats per scene (only divs that have a tei:head)
scene_divs = [d for d in divs 
              if d.find('tei:head', namespaces=ns) is not None]
scene_stats = []

for idx, div in enumerate(scene_divs, start=1):
    head = div.find('tei:head', namespaces=ns)
    title = head.text.strip() if head is not None else f"Scene {idx}"
    scene_info = {
        "scene_num": idx,
        "scene_title": title,
        "speakers": {}
    }

    for sp in div.findall('.//tei:sp', namespaces=ns):
        sp_el = sp.find('tei:speaker', namespaces=ns)
        if sp_el is None:
            continue
        raw = sp_el.text.strip()
        # strip leading/trailing punctuation so "Faust." becomes "Faust"
        speaker = raw.strip('.,;:!?…–—-"\'' + "()[]")
        stats = scene_info["speakers"].setdefault(
            speaker,
            {"lines": 0, "word_freq": Counter()}
        )

        for l in sp.findall('.//tei:l', namespaces=ns):
            text = (l.text or '').strip()
            if not text:
                continue
            stats["lines"] += 1
            doc = nlp(text)
            for tok in doc:
                if tok.is_alpha:
                    lemma = tok.lemma_.lower()
                    if lemma not in stopwords:
                        stats["word_freq"][lemma] += 1

    # convert Counter to regular dict for JSON serialization
    for s, info in scene_info["speakers"].items():
        # sort word_freq by descending count:
        info["word_freq"] = dict(
            sorted(
                info["word_freq"].items(),
                key=lambda item: item[1],
                reverse=True
            )
        )

    scene_stats.append(scene_info)

scene_stats



In [ ]:
import json

# If you haven't yet, rename the existing stats for book 1:
scene_stats_book1 = scene_stats

# After you load & process book 2 the same way, rename its stats:
# scene_stats_book2 = scene_stats

# Export both to JSON files
for stats, fname in [
    (scene_stats_book1, "book2_scene_stats.json"),
    # (scene_stats_book2, "book2_scene_stats.json"
]:
    with open(fname, "w", encoding="utf-8") as f:
        json.dump(stats, f, ensure_ascii=False, indent=2)